In [153]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
import sys
import warnings
import numpy as np
import xarray as xr
import xbudget
import regionate
import xwmt
import xwmb

sys.path.append("../examples/")
from loading import *

In [155]:
lam = "heat"

grid = load_baltic("natv", "hourly")
default_bins=True

budgets_dict = xbudget.load_preset_budget(model="MOM6_3Donly")
xbudget.collect_budgets(grid._ds, budgets_dict)
simple_budget = xbudget.aggregate(budgets_dict)

# Note: the properties of this region are quite different from the rest of the Baltic!
name = "intBaltic"
lons = np.array([8.,   20.,  29., 24.5, 24.5, 26.1, 17.5, 11.5])
lats = np.array([53.5, 53.5, 54.5,  59.,  61.,  63., 64.5,  62.])
region = regionate.GriddedRegion(name, lons, lats, grid)

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    wmb = xwmb.WaterMassBudget(
        grid,
        budgets_dict,
        region
    )
    wmb.mass_budget(lam, default_bins=default_bins)
    wmt = wmb.wmt
    wmt.load()

In [156]:
wmt.to_netcdf('../paper/data/baltic_wmb_natv_hourly_for_decompositions.nc')

In [157]:
lam_min, lam_max, dlam = -4.173, 40., 0.273519

grid_newbins = xwmt.add_gridcoords(grid, {}, {})
grid_newbins._ds = grid_newbins._ds.assign_coords({
    f"thetao_l_target" : np.arange(lam_min, lam_max, dlam),
    f"thetao_i_target" : np.arange(lam_min-dlam/2., lam_max+dlam/2, dlam),
})
grid_newbins = xwmt.add_gridcoords(
    grid_newbins,
    {"Z_target": {"center": "thetao_l_target", "outer": "thetao_i_target"}},
    {"Z_target": "extend"}
)

wmb_newbins = xwmb.WaterMassBudget(
    grid_newbins,
    budgets_dict,
    region
)
wmb_newbins.mass_budget("heat")
wmt_newbins = wmb_newbins.wmt
wmt_newbins.load();

/work/hfd/.conda/envs/xwmb/lib/python3.10/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/work/hfd/.conda/envs/xwmb/lib/python3.10/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/work/hfd/.conda/envs/xwmb/lib/python3.10/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use

In [158]:
wmt_newbins.to_netcdf('../paper/data/baltic_wmb_natv_hourly_for_decompositions_newbins.nc')

In [159]:
thetao0 = 2.5

In [160]:
boundary_fluxes = [
    'surface_exchange_flux',
    'surface_ocean_flux_advective_negative_lhs',
    'bottom_flux',
    'frazil_ice'
]

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    Gmix = wmb.map_transformations("heat", term="diffusion")['diffusion'].sel(thetao_l_target=thetao0, method="nearest").compute()
    GBC = sum([wmb.map_transformations("heat", term=term)[term] for term in boundary_fluxes]).sel(thetao_l_target=thetao0, method="nearest").compute()

In [161]:
ds = wmb.grid._ds

lam_grid = Grid(
    ds,
    coords={'lam': wmb.target_coords,},
    boundary={'lam': 'extend'},
    autoparse_metadata=False
)

dt = ds.time_bounds.diff('time_bounds').astype('float')*1.e-9
dmdt = lam_grid.interp(
    (ds.mass_density_bounds_less_than.diff('time_bounds') / dt)
    .rename({"time_bounds":"time"})
    .assign_coords({'time': ds.time}),
    "lam",
    boundary="extend"
).sel(thetao_l_target=thetao0, method="nearest").drop_vars("thetao_l_target")

/work/hfd/.conda/envs/xwmb/lib/python3.10/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


In [162]:
xr.merge([
    xr.Dataset({"Gmix":Gmix,"GBC":GBC, "dmdt":dmdt, "mask":region.mask}),
    ds[['geolon_c', 'geolat_c', 'deptho']]
]).to_netcdf("../paper/data/baltic_wmb_natv_hourly_maps.nc")

In [163]:
trans = lam_grid.interp(
    ds.convergent_mass_transport_less_than,
    "lam",
    boundary="extend"
).sel(thetao_l_target=thetao0, method="nearest").mean('time').compute()
trans_mask = (ds.convergent_mass_transport_less_than.isel(thetao_i_target=-1).mean('time')).compute()

/work/hfd/.conda/envs/xwmb/lib/python3.10/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


In [164]:
xr.Dataset({
    "region_lons": xr.DataArray(region.lons, dims=("sect_vertices",)),
    "region_lats": xr.DataArray(region.lats, dims=("sect_vertices",)),
    "convergent_transport": trans,
    "convergent_transport_mask": trans_mask.drop_vars("thetao_i_target")
}).to_netcdf("../paper/data/baltic_wmb_natv_hourly_boundary_transport.nc")